In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../dataset/raspagem-dou.csv")

In [3]:
df.head()

,index,grupo,termo_pesquisa,secao,url,titulo,resumo,data,texto_dou,origem,data_raspagem
0,9,single_group,Designar,Seção 2,https://www.in.gov.br/web/dou/-/portaria-mcom-...,"PORTARIA MCOM N° 211, DE 31 DE AGOSTO DE 2022","disposto no Decreto nº 11.164, de 8 de agosto ...",01/09/2022,"<div class=""texto-dou""> <html>\n<head></head>\...",Ministério das Comunicações,01/09/2022 11:00
1,10,single_group,Designar,Seção 2,https://www.in.gov.br/web/dou/-/portaria-mcom-...,"PORTARIA MCOM N° 199, DE 31 DE AGOSTO DE 2022","disposto no Decreto nº 11.164, de 8 de agosto ...",01/09/2022,"<div class=""texto-dou""> <html>\n<head></head>\...",Ministério das Comunicações,01/09/2022 11:00
2,11,single_group,Designar,Seção 2,https://www.in.gov.br/web/dou/-/portaria-mcom-...,"PORTARIA MCOM N° 193, DE 31 DE AGOSTO DE 2022","disposto no Decreto nº 11.164, de 8 de agosto ...",01/09/2022,"<div class=""texto-dou""> <html>\n<head></head>\...",Ministério das Comunicações,01/09/2022 11:00
3,134,single_group,Exonerar,Seção 2,https://www.in.gov.br/web/dou/-/portaria-mcom-...,"PORTARIA MCOM Nº 1, DE 4 DE JANEIRO DE 2023","disposto no Decreto nº 11.164, de 8 de agosto ...",06/01/2023,"<div class=""texto-dou""><html>\n<head></head>\n...",Ministério das Comunicações,06/01/2023 11:00
4,187,single_group,Nomear,Seção 2,https://www.in.gov.br/web/dou/-/portarias-de-1...,PORTARIAS DE 18 DE JANEIRO DE 2023,Nº 808 -<span class='highlight' style='backgro...,19/01/2023,"<p class=""dou-paragraph"">FLAVIA DUARTE NASCIME...",Casa Civil,19/01/2023 08:46


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1268 entries, 0 to 1267
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           1268 non-null   int64 
 1   grupo           1268 non-null   object
 2   termo_pesquisa  1268 non-null   object
 3   secao           1268 non-null   object
 4   url             1268 non-null   object
 5   titulo          1268 non-null   object
 6   resumo          1268 non-null   object
 7   data            1268 non-null   object
 8   texto_dou       1268 non-null   object
 9   origem          1268 non-null   object
 10  data_raspagem   1268 non-null   object
dtypes: int64(1), object(10)
memory usage: 109.1+ KB


In [5]:
df.termo_pesquisa.value_counts()

termo_pesquisa
Designar             471
Exonerar             301
Nomear               289
Dispensar            188
Tornar sem efeito     19
Name: count, dtype: int64

# limpeza de dados textuais

In [6]:
import re
import unicodedata
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

In [7]:
# Baixar recursos do NLTK (só precisa uma vez)
nltk.download("stopwords")
nltk.download("rslp")

[nltk_data] Downloading package stopwords to /Users/wesin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /Users/wesin/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [8]:
stopwords_pt = set(stopwords.words("portuguese"))
stemmer = RSLPStemmer()

In [9]:
def limpar_html(texto):
    return BeautifulSoup(texto, "html.parser").get_text()

def normalizar_acentos(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFKD', texto)
        if not unicodedata.combining(c)
    )

def limpar_texto(texto):
    """ remove caracteres indesejados e normaliza espaços usando regex"""
    texto = re.sub(r'\d+', ' ', texto) # remove números
    texto = re.sub(r'[^\w\s]', ' ', texto) # remove pontuação
    texto = re.sub(r'\s+', ' ', texto) # normaliza espaço
    return texto.strip().lower()

def remove_stopwords(tokens):
    return [t for t in tokens if t not in stopwords_pt and len(t) > 2]


def aplicar_stemming(tokens):
    return [stemmer.stem(t) for t in tokens]

def preprocessar_texto(texto, aplicar_stem=False):
    texto = str(texto)

    texto = limpar_html(texto)
    texto = normalizar_acentos(texto)
    texto = limpar_texto(texto)

    tokens = texto.split()
    tokens = remove_stopwords(tokens)

    if aplicar_stem:
        tokens = aplicar_stemming(tokens)

    return " ".join(tokens)

# preprocessamento da base (tokenização e rótulos)

In [10]:
df["texto_dou_limpo"] = df.texto_dou.apply(lambda x: preprocessar_texto(x, aplicar_stem=False))

In [11]:
textos = df.texto_dou_limpo.astype(str).tolist()
rotulos = df.termo_pesquisa.astype(str).tolist()

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_palavras = 10000
max_len = 500 # número máximo de sequencias

tokenizer = Tokenizer(num_words=num_palavras, oov_token="<OOV>")

In [13]:
tokenizer.fit_on_texts(textos)

In [14]:
sequencias = tokenizer.texts_to_sequences(textos)

In [15]:
X = pad_sequences(sequencias, maxlen=max_len, padding="post", truncating="post")

In [16]:
print(X.shape)

(1268, 500)


In [17]:
# converte saída categórica em números
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

encoder = LabelEncoder()
y = encoder.fit_transform(rotulos)

y = to_categorical(y)

print(y.shape)

(1268, 5)


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# criando o modelo

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras import Input

model = Sequential()
model.add(Input(shape=(500,)))
model.add(Embedding(input_dim=num_palavras, output_dim=128))
model.add(Bidirectional(LSTM(128, return_sequences=False)))
model.add(Dropout(0.3))
model.add(Dense(64, activation="relu"))
model.add(Dense(y.shape[1], activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

2025-11-06 15:35:21.352785: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-11-06 15:35:21.352897: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-11-06 15:35:21.352922: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-11-06 15:35:21.353158: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-06 15:35:21.353191: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 256)            │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,559,941 (5.95 MB)

 Trainable params: 1,559,941 (5.95 MB)

 Non-trainable params: 0 (0.00 B)

# trinar o modelo

In [20]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


2025-11-06 15:35:25.584164: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.4168 - loss: 1.3672 - val_accuracy: 0.3695 - val_loss: 2.4750
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.7300 - loss: 0.8442 - val_accuracy: 0.7635 - val_loss: 0.6760
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.8915 - loss: 0.3859 - val_accuracy: 0.9310 - val_loss: 0.2469
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.9618 - loss: 0.1555 - val_accuracy: 0.9557 - val_loss: 0.1753
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.9679 - loss: 0.1220 - val_accuracy: 0.9704 - val_loss: 0.1367
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.9778 - loss: 0.0721 - val_accuracy: 0.9704 - val_loss: 0.1329
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.9790 - loss: 0.0716 - val_accuracy: 0.9803 - val_loss: 0.1051
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.9815 - loss: 0.0480 - val_accuracy: 0.9852 - val_loss: 0.

# avaliar modelo

In [21]:
score = model.evaluate(X_test, y_test)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9646 - loss: 0.1625


In [22]:
import numpy as np
np.argmax(model.predict(X_test[5].reshape(1,500)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step


3

In [23]:
np.argmax(y_test[5])

3

# salvando o modelo

In [24]:
model.save("../models/modelo_dou.keras")

In [26]:
tokenizer_json = tokenizer.to_json()

In [27]:
with open("../models/tokenizer.json", "w", encoding="utf-8") as f:
    f.write(tokenizer_json)